# Interactive Molecular Dynamics with OpenMM, ASE and NanoVer

In this notebook, we run a pre-prepared OpenMM simulation of nanotube with ASE, and serve it for interactive molecular dynamics with NanoVer. 

We'll then connect a client running NGLView to it and apply some interactive forces directly from the notebook

In [2]:
from nanover.ase.openmm import ASEOpenMMRunner

## Run the NanoVer server

Let's load in the NanoVer OpenMM XML file - this file is the OpenMM System XML file, with a PDB file added to it so we can have the topology and simulation in one file.

The `ASEOpenMMRunner` class is designed to take this file and set up interactive molecular dynamics with typical settings (see [the neuraminidase example](./openmm_neuraminidase.ipynb) for an example of more advanced set up).

In [4]:
input_xml = "openmm_files/nanotube.xml"

In [5]:
# This will just automatically close previous instances of the runner if you re-run this cell multiple times
try:
    imd_runner.close()
except NameError:
    pass
imd_runner = ASEOpenMMRunner.from_xml(input_xml)

In [4]:
print(f'{imd_runner.name}: serving at {imd_runner.address}:{imd_runner.port}')

jon: NanoVer iMD Server: serving at [::]:38801


Run a few steps of MD to check it's working

In [5]:
imd_runner.run(10)

In [6]:
imd_runner.dynamics.get_number_of_steps()

10

In [7]:
imd_runner.dynamics.atoms.get_potential_energy()

28.868825418945786

Now, let's leave the dynamics running dynamics in a background thread

In [8]:
imd_runner.run()

In [9]:
imd_runner.dynamics.get_time()

29.959219104815393

The server is running, so you can connect to it in VR and you'll see something like this:

![nanotube](./images/nanover_nanotube.png)



# Visualizing the simulation with NGLView 

We have a little [script](nglclient.py) (which will eventually become part of NanoVer properly) which can show a NanoVer trajectory with [NGLView](https://github.com/arose/nglview)

To install NGLView: 

```
conda install nglview -c conda-forge
# might need: jupyter-nbextension enable nglview --py --sys-prefix

# if you already installed nglview, you can `upgrade`
conda upgrade nglview --force
# might need: jupyter-nbextension enable nglview --py --sys-prefix
```

Your browser needs to support WebGL (try Firefox or Edge)

In [10]:
from nglclient import NGLClient

In [ ]:
client = NGLClient.connect_to_single_server(port=imd_runner.port)

In [12]:
client.view

NGLWidget()

## Apply an Interactive Force 

Generally, you probably want to apply forces from the VR app, NanoVer iMD - but you can do it from python! 

First, let's apply a couple of forces to hold to nanotube in place (there are better ways to do this) 

In [13]:
from nanover.imd.particle_interaction import ParticleInteraction
import numpy as np

First, we ask the server for an ID for our interactions

In [14]:
anchor1_id = client.start_interaction()

In [15]:
anchor2_id = client.start_interaction()

Now, we can create two interactions, one at each end of the nanotube, and assign a position. 
You can repeatedly run the cell below, changing the positions or strength of the interactions. See if you can catch the methane! 

**Disclaimer**: This is *much* easier in VR.

In [16]:
first_position = np.array((10,1,0)) # nanometers!
second_position = np.array((10,1,0))
anchor1 = ParticleInteraction(position=first_position, scale=10, particles=(0,), interaction_type='spring')
anchor2 = ParticleInteraction(position=second_position, scale=10, particles=(59,), interaction_type='spring')
client.update_interaction(anchor1_id, anchor1)
client.update_interaction(anchor2_id, anchor2)

In [17]:
client.stop_interaction(anchor1_id)

True

In [18]:
client.stop_interaction(anchor2_id)

True

As always, we should clean up after ourselves:

# Close the Server

In [19]:
imd_runner.close()

# Next Steps

* The [NGLViewer notebook](nanover_nglview.ipynb) is a smaller notebook designed for visualizing your own simulations.
* Set up an OpenMM simulation with an AMBER file for a [protein-ligand system](openmm_neuraminidase.ipynb) and simulate it in NanoVer
* Set up a simulation of a [graphene sheet](openmm_graphene.ipynb) with parameters than can be controlled from the jupyter notebook. 
* Visualize an [LSD receptor in a membrane](../mdanalysis/mdanalysis_lsd.ipynb) structure with MDAnalysis and NanoVer 